# BMG Labtech CLARIOstar Plus

| Summary | Photo |
|---------|-------|
| <ul style="font-size:15px; line-height:1.6; margin-top:0;"> <li><a href="https://www.bmglabtech.com/en/clariostar-plus/" target="_blank"><b>OEM Link</b></a></li> <li><b>Communication Protocol / Hardware:</b> Serial (FTDI) / USB</li> <li><b>Communication Level:</b> Firmware</li> <li><b>Measurement Modes:</b> Absorbance, Luminescence, Fluorescence</li> <li><b>Optical Systems:</b> Dual LVF Monochromator, Physical Filters, UV/Vis Spectrometer (220–1000 nm combined range)</li> <li><b>Plate Delivery:</b> Drawer</li> <li><b>Additional Features:</b> Temperature control, Shaking, Rapid full-plate autofocus, Configurable scan modes</li> <li>VID:PID <code>0403:BB68</code></li> </ul> | <div style="width:320px; text-align:center;"> ![clariostar](img/bmg-labtech-clariostar-plus.png) <br><i>Figure: BMG Labtech CLARIOstar Plus</i> </div> |

**CLARIOstar Plus vs CLARIOstar:** The CLARIOstar Plus (post-2019) replaced the original CLARIOstar. Both share the dual LVF Monochromator + filter + spectrometer architecture. The Plus adds rapid full-plate autofocus, newer PMT options (e.g. far-red), and Voyager control software. EDR (Enhanced Dynamic Range) was introduced after 2024 and is **not** present on all CLARIOstar Plus units. This backend targets the CLARIOstar Plus but may also work with the original CLARIOstar (untested).

---
## Setup Instructions (Physical)

The CLARIOstar and CLARIOstar Plus require a minimum of two cable connections to be operational:
1. Power cord (standard IEC C13)
2. USB cable (USB-B with security screws at CLARIOstar end; USB-A at control PC end)

Optional:
If you have a plate stacking unit to use with the CLARIOstar (Plus), an additional RS-232 port is available on the CLARIOstar (Plus).

---
## Setup Instructions (Programmatic)

To control the BMG Labtech CLARIOstar Plus, create a `PlateReader` frontend
instance that uses a `CLARIOstarPlusBackend` as its backend.

Currently supported: `setup()`, `open()`, `close()`, `stop()`, status polling,
device identification (EEPROM, firmware, usage counters), and temperature control.
Measurement commands (absorbance, fluorescence, luminescence) will be added in later phases.

In [1]:
from pylabrobot import verbose

verbose(True)

In [2]:
from pylabrobot.plate_reading import PlateReader
from pylabrobot.plate_reading.bmg_labtech import CLARIOstarPlusBackend

clariostar_plus_backend = CLARIOstarPlusBackend()

pr = PlateReader(
    name="CLARIOstar",
    backend=clariostar_plus_backend,
    size_x=0.0,
    size_y=0.0,
    size_z=0.0,
)

If you have multiple FTDI devices connected, pass a `device_id` to the backend to select the correct one:

```python
clariostar_plus_backend = CLARIOstarPlusBackend(device_id="FT1234AB")
```

In [3]:
await pr.setup()

2026-02-24 12:45:27,498 - pylabrobot.io.ftdi - INFO - Successfully opened FTDI device: 430-2621
2026-02-24 12:45:27,757 - pylabrobot - INFO - read 24 bytes: 0200180c011500240000030000000000000000e00001430d
2026-02-24 12:45:27,758 - pylabrobot - WARNING - _wait_until_machine_ready called without read_timeout, waiting indefinitely
2026-02-24 12:45:27,795 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000e00001300d
2026-02-24 12:45:27,796 - pylabrobot - INFO - status: {'standby': False, 'busy': False, 'running': False, 'unread_data': False, 'initialized': True, 'drawer_open': False, 'plate_detected': False, 'temperature_bottom': None, 'temperature_top': None, 'heating_active': True}
2026-02-24 12:45:27,869 - pylabrobot - INFO - read 271 bytes: 02010f0c070500240000000100000a0101010100000100ee0200000f00e2030000000000000304000001000001020000000000000000000032000000000000000000000000000000000000000074006f0000000000000065000000dc050000000000000000f4010803a70408076009da

```{note}
Expected behaviour: the machine performs its initialization routine (FTDI connection, baudrate configuration,
initialize command). The backend polls status until the `initialized` flag is set.
```

---
### Defining & Assigning a Plate

Every measurement requires a plate to be assigned to the plate reader (to tell the firmware what the positions for reading are).

In [4]:
from pylabrobot.resources.corning import Cor_96_wellplate_360ul_Fb

plate = Cor_96_wellplate_360ul_Fb("test_plate")
pr.assign_child_resource(plate)

---
## Usage: Drawer

The CLARIOstar loads and unloads plates via a motorized drawer.

In [5]:
# Open the drawer (plate out)
await pr.open()

2026-02-24 12:45:27,983 - pylabrobot - INFO - read 24 bytes: 0200180c012500240000030000000000000000e00001530d
2026-02-24 12:45:27,984 - pylabrobot - WARNING - _wait_until_machine_ready called without read_timeout, waiting indefinitely
2026-02-24 12:45:28,021 - pylabrobot - INFO - read 24 bytes: 0200180c012500240000030000000000000000e00001530d
2026-02-24 12:45:28,022 - pylabrobot - INFO - status: {'standby': False, 'busy': True, 'running': False, 'unread_data': False, 'initialized': True, 'drawer_open': False, 'plate_detected': False, 'temperature_bottom': None, 'temperature_top': None, 'heating_active': True}
2026-02-24 12:45:28,160 - pylabrobot - INFO - read 24 bytes: 0200180c012500240000030000000000000000e00001530d
2026-02-24 12:45:28,161 - pylabrobot - INFO - status: {'standby': False, 'busy': True, 'running': False, 'unread_data': False, 'initialized': True, 'drawer_open': False, 'plate_detected': False, 'temperature_bottom': None, 'temperature_top': None, 'heating_active': True}
2

In [6]:
# Place your plate on the drawer (manually or via a robotic arm), then close it (plate in)
await pr.close()

2026-02-24 12:45:32,209 - pylabrobot - INFO - read 24 bytes: 0200180c012500210000030000000000000000e00001500d
2026-02-24 12:45:32,210 - pylabrobot - WARNING - _wait_until_machine_ready called without read_timeout, waiting indefinitely
2026-02-24 12:45:32,252 - pylabrobot - INFO - read 24 bytes: 0200180c012500210000030000000000000000e00001500d
2026-02-24 12:45:32,253 - pylabrobot - INFO - status: {'standby': False, 'busy': True, 'running': False, 'unread_data': False, 'initialized': True, 'drawer_open': True, 'plate_detected': False, 'temperature_bottom': None, 'temperature_top': None, 'heating_active': True}
2026-02-24 12:45:32,390 - pylabrobot - INFO - read 24 bytes: 0200180c012500210000030000000000000000e00001500d
2026-02-24 12:45:32,390 - pylabrobot - INFO - status: {'standby': False, 'busy': True, 'running': False, 'unread_data': False, 'initialized': True, 'drawer_open': True, 'plate_detected': False, 'temperature_bottom': None, 'temperature_top': None, 'heating_active': True}
202

---
## Device Identification

EEPROM configuration, firmware info, usage counters, and status queries.

In [7]:
config = clariostar_plus_backend.configuration
for key, value in config.items():
    print(f"  {key:25s} {value}")

print()
modes = await clariostar_plus_backend.request_available_detection_modes()
print(f"  Available detection modes: {', '.join(modes) if modes else 'none'}")

2026-02-24 12:45:40,268 - pylabrobot - INFO - read 271 bytes: 02010f0c070500240000000100000a0101010100000100ee0200000f00e2030000000000000304000001000001020000000000000000000032000000000000000000000000000000000000000074006f0000000000000065000000dc050000000000000000f4010803a70408076009da08ac0d0000000000000000000000000000000000000000000000000100000001010000000000000001010000000000000012029806ae013d0a4605ee01fbff700c00000000a40058ff8e03f20460ff5511fe0b55118f1a170298065aff970668042603bc14b804080791009001463228460a0046071e0000000000000000002103d40628002c01900146001e00001411001209ac0d6009000000000020260d
2026-02-24 12:45:40,270 - pylabrobot - INFO - EEPROM: 263 bytes, head=070500240000000100000a0101010100


  serial_number             430-2621
  firmware_version          1.35
  firmware_build_timestamp  Nov 20 2020 11:51:21
  model_name                CLARIOstar Plus
  machine_type_code         36
  max_temperature           45
  has_absorbance            True
  has_fluorescence          True
  has_luminescence          True
  has_alpha_technology      True
  excitation_monochromator_max_nm 750
  emission_monochromator_max_nm 994
  excitation_filter_slots   4
  dichroic_filter_slots     3
  emission_filter_slots     4

  Available detection modes: absorbance, absorbance_spectrum, fluorescence, luminescence, alpha_technology


In [8]:
counters = await clariostar_plus_backend.request_usage_counters()
for name, value in counters.items():
    print(f"  {name:25s} {value:>12,}")

2026-02-24 12:45:40,327 - pylabrobot - INFO - read 50 bytes: 0200320c210500240000001e023d0000078c000004e7000003d9000275cf000012fa0000000a0000000a0000000a0005b10d


  flashes                      1,966,653
  testruns                         1,932
  wells                          125,500
  well_movements                  98,500
  active_time_s                  161,231
  shake_time_s                     4,858
  pump1_usage                         10
  pump2_usage                         10
  alpha_time                          10


In [9]:
status = await clariostar_plus_backend.request_machine_status()
for flag, value in status.items():
    print(f"  {flag:20s} {value}")

2026-02-24 12:45:40,373 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000e00001300d


  standby              False
  busy                 False
  running              False
  unread_data          False
  initialized          True
  drawer_open          False
  plate_detected       False
  temperature_bottom   None
  temperature_top      None
  heating_active       True


In [10]:
if await clariostar_plus_backend.sense_plate_present():
    print("Plate is in the drawer")
else:
    print("No plate detected")

if await clariostar_plus_backend.is_ready():
    print("Machine is ready")
else:
    print("Machine is busy")

2026-02-24 12:45:40,424 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000e00001300d
2026-02-24 12:45:40,465 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000e00001300d


No plate detected
Machine is ready


---
## Temperature

The CLARIOstar Plus has a dual-plate incubator (bottom + top heating elements) with
0.1 °C resolution. The firmware exposes three temperature modes via command family `0x06`:

| Mode | Payload | Effect |
|------|---------|--------|
| **OFF** | `0x0000` | Disable heating and sensor readout |
| **MONITOR** | `0x0001` | Activate sensors only (no heating) |
| **SET** | target x 10 | Heat to target °C and activate sensors |

```{important}
The firmware treats all three as a single-state register: each new `0x06` command
**replaces** the previous one. `measure_temperature()` is safe to call while heating
is active -- it calls `start_temperature_monitoring()` which is idempotent (checks
`request_temperature_monitoring_on()` and returns immediately if sensors are already
reporting), so it will never overwrite an active heating setpoint.
```

### Reading the current temperature

`measure_temperature()` returns the current plate temperature. On the first call it
activates the sensors via `start_temperature_monitoring()` (~200 ms warmup); subsequent
calls detect that sensors are already reporting and return instantly.

In [11]:
active = await clariostar_plus_backend.request_temperature_monitoring_on()
print(f"Temperature monitoring on: {active}")

2026-02-24 12:45:40,510 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000e00001300d


Temperature monitoring on: False


In [12]:
active = await clariostar_plus_backend.request_temperature_control_on()
print(f"Temperature control on: {active}")

2026-02-24 12:45:40,553 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000e00001300d


Temperature control on: True


In [13]:
# Read each sensor individually, or the mean of both
bottom = await clariostar_plus_backend.measure_temperature(sensor="bottom")
top = await clariostar_plus_backend.measure_temperature(sensor="top")
mean = await clariostar_plus_backend.measure_temperature(sensor="mean")

print(f"Bottom plate: {bottom:.1f} °C")
print(f"Top plate:    {top:.1f} °C")
print(f"Mean:         {mean:.1f} °C")

2026-02-24 12:45:40,596 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000e00001300d
2026-02-24 12:45:40,636 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000e00001300d
2026-02-24 12:45:40,674 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000e00001300d
2026-02-24 12:45:40,712 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000e00001300d
2026-02-24 12:45:40,750 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000e00001300d
2026-02-24 12:45:40,788 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000e00001300d
2026-02-24 12:45:40,826 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000e00001300d
2026-02-24 12:45:40,864 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000e00001300d
2026-02-24 12:45:40,902 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000001500156e00001d80d
2026-02-24

Bottom plate: 33.6 °C
Top plate:    34.2 °C
Mean:         33.9 °C


### Checking if temperature monitoring is on

`request_temperature_monitoring_on()` returns `True` when the temperature sensors
are reporting values (i.e. either heating or monitoring is enabled).

In [14]:
active = await clariostar_plus_backend.request_temperature_monitoring_on()
print(f"Temperature monitoring on: {active}")

2026-02-24 12:45:41,099 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000001500156e00001d80d


Temperature monitoring on: True


In [15]:
active = await clariostar_plus_backend.request_temperature_control_on()
print(f"Temperature control on: {active}")

2026-02-24 12:45:41,144 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000001500156e00001d80d


Temperature control on: True


### Heating to a target temperature

`start_temperature_control(target)` heats the incubator to the target °C.
Use `measure_temperature()` to monitor progress -- it is safe to call during
active heating (see note above).

When done:
- `stop_temperature_control()` stops heating but **keeps sensors active** (downgrades to MONITOR).
- `stop_temperature_monitoring()` stops **both** heating and sensors (sends OFF).

In [16]:
import asyncio

# Start heating to 37 °C
await clariostar_plus_backend.start_temperature_control(37.0)

# Monitor until we reach the setpoint (or close enough)
for _ in range(35):
    temp = await clariostar_plus_backend.measure_temperature()
    print(f"Bottom plate: {temp:.1f} °C")
    if temp >= 36.5:
        print("Reached target range!")
        break
    await asyncio.sleep(2)

# Stop heating but keep sensors active
await clariostar_plus_backend.stop_temperature_control()
print("Heating stopped (sensors still active).")

2026-02-24 12:45:41,202 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000e00001300d
2026-02-24 12:45:41,541 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000001500156e00001d80d
2026-02-24 12:45:41,579 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000001500156e00001d80d


Bottom plate: 33.6 °C


2026-02-24 12:45:43,618 - pylabrobot - INFO - read 24 bytes: 0200180c0105002400000000000000014f0155e00001d60d
2026-02-24 12:45:43,656 - pylabrobot - INFO - read 24 bytes: 0200180c0105002400000000000000014f0155e00001d60d


Bottom plate: 33.5 °C


2026-02-24 12:45:45,697 - pylabrobot - INFO - read 24 bytes: 0200180c0105002400000000000000014f0155e00001d60d
2026-02-24 12:45:45,735 - pylabrobot - INFO - read 24 bytes: 0200180c0105002400000000000000014f0155e00001d60d


...

Bottom plate: 35.9 °C


2026-02-24 12:46:52,185 - pylabrobot - INFO - read 24 bytes: 0200180c01050024000000000000000168016fc00001e90d
2026-02-24 12:46:52,223 - pylabrobot - INFO - read 24 bytes: 0200180c01050024000000000000000168016fc00001e90d


Bottom plate: 36.0 °C


2026-02-24 12:46:54,264 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d


Heating stopped (sensors still active).


In [17]:
# After stop_temperature_control(), sensors are still active (MONITOR mode)
active = await clariostar_plus_backend.request_temperature_monitoring_on()
print(f"Temperature monitoring on: {active}")  # True -- sensors still reporting

# Full shutdown: stop sensors too
await clariostar_plus_backend.stop_temperature_monitoring()
active = await clariostar_plus_backend.request_temperature_monitoring_on()
print(f"Temperature monitoring on: {active}")  # False -- everything off

2026-02-24 12:46:54,310 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 12:46:54,312 - pylabrobot - WARNING - stop_temperature_monitoring sends OFF -- this also disables heating
2026-02-24 12:46:54,353 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 12:46:54,393 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d


Temperature monitoring on: False
Temperature monitoring on: False


In [18]:
active = await clariostar_plus_backend.request_temperature_control_on()
print(f"Temperature control on: {active}")

2026-02-24 12:46:54,452 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d


Temperature control on: False


---
## Planned Features

The sections below cover measurement modes that are not yet implemented.
Code cells are commented out and will be uncommented as each feature lands.

### Absorbance

Single-wavelength, multi-wavelength, transmittance, partial well selection.

In [19]:
# results = await pr.read_absorbance(
#     wavelength=600,
# )
#
# print(f"OD at 600nm, well A1: {results[0]['data'][0][0]}")

In [20]:
# results = await pr.read_absorbance(
#     wavelength=450,
#     report="transmittance",
# )
#
# # Transmittance is in percent (0-100)
# print(f"Transmittance at 450nm, well A1: {results[0]['data'][0][0]}%")

In [21]:
# results = await pr.read_absorbance(
#     wavelength=600,
#     wavelengths=[260, 280, 450, 600, 750],
# )
#
# for r in results:
#     print(f"Wavelength {r['wavelength']} nm -> A1 OD: {r['data'][0][0]:.4f}")

In [22]:
# column_1_wells = [plate.get_well(f"{row}1") for row in "ABCDEFGH"]
#
# results = await pr.read_absorbance(
#     wavelength=600,
#     wells=column_1_wells,
# )

### Fluorescence

Basic, custom gain/bandwidth, bottom-optic.

In [23]:
# results = await pr.read_fluorescence(
#     excitation_wavelength=485,
#     emission_wavelength=528,
#     focal_height=8.5,
# )
#
# print(f"GFP fluorescence, well A1: {results[0]['data'][0][0]}")

In [24]:
# results = await pr.read_fluorescence(
#     excitation_wavelength=485,
#     emission_wavelength=528,
#     focal_height=8.5,
#     gain=1500,
#     ex_bandwidth=10,
#     em_bandwidth=20,
#     flashes=50,
# )

In [25]:
# results = await pr.read_fluorescence(
#     excitation_wavelength=544,
#     emission_wavelength=590,
#     focal_height=4.5,
#     bottom_optic=True,
# )

### Luminescence

Basic luminescence, partial well selection.

In [26]:
# results = await pr.read_luminescence(
#     focal_height=13.0,
# )
#
# print(f"Temperature: {results[0]['temperature']:.1f} °C")
# print(f"Well A1 RLU: {results[0]['data'][0][0]}")

In [27]:
# selected_wells = [plate.get_well("A1"), plate.get_well("D6"), plate.get_well("H12")]
#
# results = await pr.read_luminescence(
#     focal_height=13.0,
#     wells=selected_wells,
# )

### Advanced Features

Scan modes, shaking, non-blocking reads, combined features.

In [28]:
# from pylabrobot.plate_reading.bmg_labtech.clariostar_plus_backend import StartCorner
#
# results = await pr.read_absorbance(
#     wavelength=600,
#     start_corner=StartCorner.BOTTOM_RIGHT,
#     unidirectional=True,
#     vertical=True,
# )

In [29]:
# from pylabrobot.plate_reading.bmg_labtech.clariostar_plus_backend import ShakerType
#
# results = await pr.read_luminescence(
#     focal_height=13.0,
#     shake_type=ShakerType.ORBITAL,
#     shake_speed_rpm=300,
#     shake_duration_s=5,
# )

In [30]:
# import asyncio
#
# await pr.read_absorbance(
#     wavelength=600,
#     wavelengths=[260, 280, 450, 600],
#     wait=False,
# )
#
# status = await clariostar_plus_backend.request_machine_status()
# while not status["unread_data"]:
#     print("Measuring...")
#     await asyncio.sleep(1)
#     status = await clariostar_plus_backend.request_machine_status()
#
# results = await clariostar_plus_backend.collect_absorbance_measurement(plate)

In [31]:
# row_A = [plate.get_well(f"A{col}") for col in range(1, 13)]
#
# results = await pr.read_fluorescence(
#     excitation_wavelength=544,
#     emission_wavelength=590,
#     focal_height=6.0,
#     wells=row_A,
#     gain=2000,
#     shake_type=ShakerType.ORBITAL,
#     shake_speed_rpm=300,
#     shake_duration_s=3,
#     start_corner=StartCorner.BOTTOM_RIGHT,
# )

---
## Reference: How Absorbance Calculation Works

The machine does not measure absorbance directly. It measures **raw detector counts**
and the backend converts them to transmittance and then to OD.
See `DESIGN.md` in `pylabrobot/plate_reading/bmg_labtech/` for the full protocol reference.

---
## Closing Connection

In [32]:
pr.unassign_child_resource(plate)

In [33]:
await pr.stop()

2026-02-24 12:46:54,855 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 12:46:54,896 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d
2026-02-24 12:46:54,938 - pylabrobot - INFO - read 24 bytes: 0200180c010500240000000000000000000000c00001100d


This closes the FTDI connection. After calling `stop()`, you must call `setup()` again before using the plate reader.